### 1. 전 세계 IATA 코드와 도시 이름 매핑
OpenFlights 데이터셋을 기반으로 전 세계 IATA 코드와 도시 이름 매핑을 생성합니다.
IATA 코드를 도시 이름으로 변환할 수 있는 매핑 테이블을 구축합니다.

In [2]:
import pandas as pd

def load_airport_mapping(file_path):
    """
    OpenFlights 데이터셋에서 IATA 코드와 도시 이름 매핑 생성.
    """
    columns = ["Airport ID", "Name", "City", "Country", "IATA", "ICAO", "Latitude",
               "Longitude", "Altitude", "Timezone", "DST", "Tz database time zone", "Type", "Source"]
    airport_data = pd.read_csv(file_path, header=None, names=columns)
    # IATA 코드와 도시 이름 매핑 생성
    return airport_data[["IATA", "City"]].dropna().set_index("IATA").to_dict()["City"]

# OpenFlights 데이터셋 경로
file_path_airports = "/home/gyuha_lee/Opensource_dev/get_information/mine/airports.dat"

# OpenFlights 데이터셋을 기반으로 매핑 생성
airport_city_mapping = load_airport_mapping(file_path_airports)

# 매핑 확인
print(f"총 매핑된 IATA 코드: {len(airport_city_mapping)}")


총 매핑된 IATA 코드: 6034


### 2. 사용자 입력 받기
사용자로부터 필요한 정보를 입력받습니다. 입력값은 이후 데이터 필터링 및 추천 점수 계산에 사용됩니다.


In [3]:
def get_user_input():
    print("사용자 정보를 입력하세요:")
    
    seat_type = input("탑승 클래스 (Economy Class, Premium Economy, Business Class, First Class): ").strip()
    route = input("여행 구간 (예: Dubai to London): ").strip()
    analyze_reviews = input("리뷰 분석 여부 (Yes/No): ").strip().lower() == "yes"
    
    print("서비스 선호도를 입력하세요 (1~5 순위):")
    preferences = {
        "Seat Comfort": int(input("좌석 편안함 (1~5): ").strip()),
        "Cabin Staff Service": int(input("객실 승무원 서비스 (1~5): ").strip()),
        "Food & Beverages": int(input("기내 음식 및 음료 (1~5): ").strip()),
        "Inflight Entertainment": int(input("기내 엔터테인먼트 (1~5): ").strip()),
        "Ground Service": int(input("지상 직원 서비스 (1~5): ").strip()),
        "Wifi & Connectivity": int(input("기내 와이파이 (1~5): ").strip()),
        "Value For Money": int(input("가격 대비 가치 (1~5): ").strip()),
    }
    
    return {
        "seat_type": seat_type,
        "route": route,
        "analyze_reviews": analyze_reviews,
        "preferences": preferences,
    }

user_input = get_user_input()


사용자 정보를 입력하세요:
서비스 선호도를 입력하세요 (1~5 순위):


### 3. 데이터 로드 및 전처리
JSON 데이터셋을 로드한 후, 필요한 열만 유지하고 불필요한 데이터를 제거합니다.


In [5]:
import json

def normalize_route_with_mapping(route, mapping):
    """
    Route 값을 OpenFlights 매핑 데이터를 활용해 도시 이름으로 통일.
    """
    if pd.isnull(route) or isinstance(route, (int, float)):
        return ""
    if "via" in route.lower():
        route = route.split("via")[0].strip()
    route_parts = route.lower().replace("to", "").split()

    # IATA 코드 또는 도시 이름을 도시 이름으로 통일
    normalized_parts = [mapping.get(part.upper(), part) for part in route_parts]

    # 출발지와 도착지가 존재하는 경우 " to " 형식으로 반환
    if len(normalized_parts) == 2:
        return " to ".join(normalized_parts).strip()
    return ""


def preprocess_data(file_path, mapping):
    """
    JSON 데이터셋을 전처리하고 필요한 필드만 유지.
    """
    # JSON 데이터 로드
    with open(file_path, 'r') as file:
        data = json.load(file)
    df = pd.DataFrame(data)

    # 불필요한 열 제거
    unnecessary_columns = ["Author"]
    df.drop(columns=unnecessary_columns, inplace=True, errors='ignore')

    # Route 정규화: 도시 이름으로 통일
    df["Normalized Route"] = df["Route"].apply(lambda r: normalize_route_with_mapping(r, mapping))

    # 결측치 처리: 유효하지 않은 Route 제거
    df = df[df["Normalized Route"] != ""]

    # 필요한 열만 유지
    required_columns = [
        "Airline Name", "Normalized Route", "Seat Type", "Seat Comfort",
        "Cabin Staff Service", "Food & Beverages", "Inflight Entertainment",
        "Ground Service", "Wifi & Connectivity", "Value For Money", "Review Content"
    ]
    df = df[required_columns]
    return df

file_path_reviews = "/home/gyuha_lee/Opensource_dev/get_information/mine/all_airline_reviews.json"
df = preprocess_data(file_path_reviews, airport_city_mapping)


### 4. Route 정규화 및 필터링
사용자 입력값과 데이터셋 간의 일관성을 확보하기 위해 Route를 정규화하고 데이터를 필터링합니다.


In [6]:
def normalize_route_with_mapping(route, mapping):
    """
    Route 값을 OpenFlights 매핑 데이터를 활용해 도시 이름으로 통일.
    """
    if pd.isnull(route) or isinstance(route, (int, float)):
        return ""
    if "via" in route.lower():
        route = route.split("via")[0].strip()
    route_parts = route.lower().replace("to", "").split()

    # IATA 코드 또는 도시 이름을 도시 이름으로 통일
    normalized_parts = [mapping.get(part.upper(), part) for part in route_parts]

    # 출발지와 도착지가 존재하는 경우 " to " 형식으로 반환
    if len(normalized_parts) == 2:
        return " to ".join(normalized_parts).strip()
    return ""

def match_route(input_route, data_route):
    """
    표준화된 도시 이름을 기반으로 Route를 비교.
    """
    try:
        input_start, input_end = input_route.split(" to ")
        data_start, data_end = data_route.split(" to ")
        return (input_start == data_start and input_end == data_end) or \
               (input_start == data_end and input_end == data_start)
    except ValueError:
        return False

def filter_data(df, user_input):
    """
    사용자 입력값에 따라 데이터를 필터링합니다.
    """
    seat_type = user_input["seat_type"].strip().lower()
    input_route = user_input["route"].strip()

    # 입력 Route 정규화
    normalized_input_route = normalize_route_with_mapping(input_route, airport_city_mapping)

    # 탑승 클래스 필터링
    filtered_df = df[df["Seat Type"].str.strip().str.lower() == seat_type]

    # 여행 구간 필터링
    filtered_df = filtered_df[filtered_df["Normalized Route"].apply(lambda r: match_route(normalized_input_route, r))]

    return filtered_df

filtered_data = filter_data(df, user_input)

if filtered_data.empty:
    print("조건에 맞는 데이터가 없습니다. 입력 조건을 확인하세요.")
else:
    print("필터링된 데이터:")
    print(filtered_data.head(100))


필터링된 데이터:
                         Airline Name Normalized Route      Seat Type  \
739         Aeroflot Russian Airlines  london to dubai  Economy Class   
4069                       Air France  london to dubai  Economy Class   
11423             Azerbaijan Airlines  dubai to london  Economy Class   
11430             Azerbaijan Airlines  dubai to london  Economy Class   
11432             Azerbaijan Airlines  dubai to london  Economy Class   
18355                        Egyptair  london to dubai  Economy Class   
18405                        Egyptair  london to dubai  Economy Class   
18932                        Emirates  dubai to london  Economy Class   
19096                        Emirates  london to dubai  Economy Class   
19192                        Emirates  london to dubai  Economy Class   
19207                        Emirates  dubai to london  Economy Class   
23770                        Gulf Air  london to dubai  Economy Class   
23771                        Gulf Air  lo

### 5. 리뷰 감성 분석
BERT 모델을 사용하여 리뷰 텍스트를 분석하고 감성 점수를 추가합니다.


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def analyze_review_sentiment_pytorch(df, analyze_reviews):
    """
    필터링된 데이터를 대상으로 리뷰 감성 분석 수행.
    """
    if not analyze_reviews:
        # 리뷰 분석이 비활성화된 경우 원본 데이터 반환
        return df

    # GPU 또는 CPU 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # BERT 모델 로드
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.to(device)

    def predict_sentiment(review):
        """
        단일 리뷰 텍스트에 대해 감성 점수를 예측.
        """
        if pd.isnull(review) or not review.strip():
            return 0  # 빈 리뷰는 0점 처리
        # BERT 입력 데이터 생성
        inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        # 모델 출력
        outputs = model(**inputs)
        # 감성 점수 예측
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return torch.argmax(probs, dim=-1).item() + 1  # 점수는 1~5 범위

    # 진행률 출력용 변수
    total_reviews = len(df)
    print(f"총 리뷰 개수: {total_reviews}")

    # 리뷰 감성 점수 추가
    sentiment_scores = []
    for idx, review in enumerate(df["Review Content"]):
        sentiment_scores.append(predict_sentiment(review))
        # 진행률 업데이트
        progress = (idx + 1) / total_reviews * 100
        print(f"\r진행률: {progress:.2f}% ({idx + 1}/{total_reviews})", end="")

    print("\n리뷰 감성 분석 완료!")
    df["Sentiment Score"] = sentiment_scores
    return df

# 필터링된 데이터에 대해 감성 분석 수행
filtered_data = analyze_review_sentiment_pytorch(filtered_data, user_input["analyze_reviews"])


총 리뷰 개수: 34
진행률: 100.00% (34/34)
리뷰 감성 분석 완료!


### 6. 추천 점수 계산
사용자 선호도와 감성 분석 점수를 기반으로 추천 점수를 계산합니다.


In [8]:
def calculate_recommendation_score(df, user_input):
    preferences = user_input["preferences"]
    total_weight = sum(preferences.values())
    weights = {k: v / total_weight for k, v in preferences.items()}
    df["Recommendation Score"] = (
        df["Seat Comfort"] * weights["Seat Comfort"] +
        df["Cabin Staff Service"] * weights["Cabin Staff Service"] +
        df["Food & Beverages"] * weights["Food & Beverages"] +
        df["Inflight Entertainment"] * weights["Inflight Entertainment"] +
        df["Ground Service"] * weights["Ground Service"] +
        df["Wifi & Connectivity"] * weights["Wifi & Connectivity"] +
        df["Value For Money"] * weights["Value For Money"] +
        df["Sentiment Score"]
    )
    return df

scored_data = calculate_recommendation_score(filtered_data, user_input)


### 7. 항공사 추천
추천 점수를 기반으로 상위 항공사를 사용자에게 제안합니다.


In [13]:
def recommend_airlines(df):
    airline_scores = df.groupby("Airline Name")["Recommendation Score"].mean().sort_values(ascending=False)
    return airline_scores.head(100)

recommendations = recommend_airlines(scored_data)
print("추천 항공사:")
print(recommendations)


추천 항공사:
Airline Name
Royal Brunei Airlines             6.817460
Azerbaijan Airlines               6.714286
Gulf Air                          6.053571
Aeroflot Russian Airlines         5.523810
Mea Middle East Airlines          4.714286
Kuwait Airways                    3.523810
Emirates                          3.428571
Saudi Arabian Airlines            3.285714
Turkish Airlines                  3.285714
Egyptair                          2.452381
Air France                        2.095238
Pegasus Airlines                  2.000000
Virgin Atlantic Airways           2.000000
Ukraine International Airlines    1.888889
Name: Recommendation Score, dtype: float64
